In [1]:
import time

import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)
# pd.reset_option(‘max_columns’)

In [3]:
# EVENTMSGACTIONTYPE
# ---
# 1 - Jumpshot
# 2 - Lost ball Turnover
# 3 - ?
# 4 - Traveling Turnover / Off Foul
# 5 - Layup
# 7 - Dunk
# 10 - Free throw 1-1
# 11 - Free throw 1-2
# 12 - Free throw 2-2
# 40 - out of bounds
# 41 - Block/Steal
# 42 - Driving Layup
# 50 - Running Dunk
# 52 - Alley Oop Dunk
# 55 - Hook Shot
# 57 - Driving Hook Shot
# 58 - Turnaround hook shot
# 66 - Jump Bank Shot
# 71 - Finger Roll Layup
# 72 - Putback Layup
# 108 - Cutting Dunk Shot

In [4]:
# EVENTMSGTYPE
# ---
# 1 - Make
# 2 - Miss
# 3 - Free Throw
# 4 - Rebound
# 5 - out of bounds / Turnover / Steal
# 6 - Personal Foul
# 7 - Violation
# 8 - Substitution
# 9 - Timeout
# 10 - Jumpball
# 12 - Start Q1?
# 13 - Start Q2?

In [5]:
import glob

df = pd.DataFrame()

pbp_files = glob.glob('../utils/pbp_data/*.csv')

for pbp_file in pbp_files:
    season = pbp_file.split('/')[-1].split('.')[0]
    print(season)
    season_df = pd.read_csv(pbp_file)
    season_df['SEASON'] = season
    
    df = pd.concat([df, season_df])

2023-24
2024-25
2022-23


In [6]:
df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,PCTIMESECONDS,SEASON
0,22301198,2,12,0,1,3:42 PM,12:00,NaN,Start of 1st Period (3:42 PM EST),NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,2023-24
1,22301198,4,10,0,1,3:42 PM,12:00,Jump Ball Jackson-Davis vs. Yurtseven: Tip to ...,NaN,NaN,NaN,NaN,4.0,1631218,Trayce Jackson-Davis,1.610613e+09,Golden State,Warriors,GSW,5,1630209,Omer Yurtseven,1.610613e+09,Utah,Jazz,UTA,5,1630548,Johnny Juzang,1.610613e+09,Utah,Jazz,UTA,1,0,2023-24
2,22301198,7,1,3,1,3:42 PM,11:39,NaN,NaN,Yurtseven 5' Hook Shot (2 PTS) (Hendricks 1 AST),2 - 0,-2,5.0,1630209,Omer Yurtseven,1.610613e+09,Utah,Jazz,UTA,5,1641707,Taylor Hendricks,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,21,2023-24
3,22301198,9,1,80,1,3:42 PM,11:22,Wiggins 21' Step Back Jump Shot (2 PTS) (Paul ...,NaN,NaN,2 - 2,TIE,4.0,203952,Andrew Wiggins,1.610613e+09,Golden State,Warriors,GSW,4,101108,Chris Paul,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,1,38,2023-24
4,22301198,10,5,6,1,3:43 PM,11:02,NaN,NaN,Yurtseven Double Dribble Turnover (P1.T1),NaN,NaN,5.0,1630209,Omer Yurtseven,1.610613e+09,Utah,Jazz,UTA,0,0,NaN,NaN,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1,58,2023-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574413,22200001,637,5,2,4,10:07 PM,0:32,Brown Lost Ball Turnover (P4.T11),NaN,Harris STEAL (3 STL),NaN,NaN,4.0,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,5,202699,Tobias Harris,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,1,688,2022-23
574414,22200001,639,1,50,4,10:07 PM,0:29,NaN,NaN,Harris 1' Running Dunk (18 PTS),117 - 126,9,5.0,202699,Tobias Harris,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,691,2022-23
574415,22200001,640,2,1,4,10:07 PM,0:06,MISS Horford 25' 3PT Jump Shot,NaN,NaN,NaN,NaN,4.0,201143,Al Horford,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,714,2022-23
574416,22200001,641,4,0,4,10:07 PM,0:04,NaN,NaN,Harden REBOUND (Off:0 Def:8),NaN,NaN,5.0,201935,James Harden,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1,716,2022-23


In [7]:
def get_description(row):
    if pd.isna(row.HOMEDESCRIPTION):
        return row.VISITORDESCRIPTION
    else:
        return row.HOMEDESCRIPTION

In [8]:
import pandas as pd
import re

# Function to identify if the play is a 3-point shot
def parse_shot_description(text):
    if pd.isnull(text):  # Check for NaN values
        return None
    pattern = r"\b3PT\b"
    if re.search(pattern, text):
        return '3P'
    else:
        return '2P'

In [9]:
streaks_df = pd.DataFrame()
nan_added = False  # Flag to track if a NaN row has been added
player_id = 201939

df['SHOT_TYPE'] = df['HOMEDESCRIPTION'].combine_first(df['VISITORDESCRIPTION']).apply(parse_shot_description)
df['is_three_pointer'] = (df['SHOT_TYPE'] == '3P')


gsw_games = df[(df.PLAYER1_TEAM_ABBREVIATION == 'GSW') | (df.PLAYER2_TEAM_ABBREVIATION == 'GSW')]


for game_id, game_df in tqdm(gsw_games.groupby('GAME_ID')):
    for _, row in game_df.iterrows():
        # check if there is a shot made in the row
        if row.EVENTMSGTYPE == 1:
            # if there was check did Steph score a 3
            if row.PLAYER1_ID == player_id and row.is_three_pointer:
                # if he did increase the streak by 1
                streaks_df = pd.concat([streaks_df, pd.DataFrame([row])], ignore_index=True)
                nan_added = False  # Reset flag since a streak continues
            # if a teammate scored reset the streak (teammate has the same PLAYER1_TEAM_ID)
            elif row.PLAYER1_ID != player_id and row.PLAYER1_TEAM_ABBREVIATION == 'GSW':
                # Add a single NaN row if it hasn't been added yet
                if not nan_added:
                    streaks_df = pd.concat([streaks_df, pd.DataFrame([pd.Series(dtype='object', index=streaks_df.columns)])], ignore_index=True)
                    nan_added = True  # Mark that a NaN has been added
            # if opponent scored continue
            elif row.PLAYER1_TEAM_ABBREVIATION != 'GSW':
                continue

100%|██████████| 176/176 [00:02<00:00, 86.58it/s]


In [10]:
streaks_df

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,PCTIMESECONDS,SEASON,SHOT_TYPE,is_three_pointer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22200002.0,525.0,1.0,2.0,3.0,12:26 AM,5:27,Curry 24' 3PT Running Jump Shot (21 PTS) (Wigg...,NaN,NaN,64 - 84,20,4.0,201939.0,Stephen Curry,1.610613e+09,Golden State,Warriors,GSW,4.0,203952.0,Andrew Wiggins,1.610613e+09,Golden State,Warriors,GSW,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,393.0,2022-23,3P,True
2,22200002.0,535.0,1.0,79.0,3.0,12:28 AM,4:51,Curry 28' 3PT Pullup Jump Shot (24 PTS),NaN,NaN,64 - 87,23,4.0,201939.0,Stephen Curry,1.610613e+09,Golden State,Warriors,GSW,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,429.0,2022-23,3P,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22200002.0,721.0,1.0,79.0,4.0,1:04 AM,4:01,Curry 25' 3PT Pullup Jump Shot (27 PTS) (DiVin...,NaN,NaN,93 - 109,16,4.0,201939.0,Stephen Curry,1.610613e+09,Golden State,Warriors,GSW,4.0,1628978.0,Donte DiVincenzo,1.610613e+09,Golden State,Warriors,GSW,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,479.0,2022-23,3P,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,22400207.0,460.0,1.0,79.0,3.0,8:42 PM,4:28,NaN,NaN,Curry 28' 3PT Pullup Jump Shot (28 PTS),95 - 73,-22,5.0,201939.0,Stephen Curry,1.610613e+09,Golden State,Warriors,GSW,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,452.0,2024-25,3P,True
1215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,22400207.0,650.0,1.0,1.0,4.0,9:21 PM,3:23,NaN,NaN,Curry 26' 3PT Jump Shot (31 PTS) (Green 11 AST),119 - 108,-11,5.0,201939.0,Stephen Curry,1.610613e+09,Golden State,Warriors,GSW,5.0,203110.0,Draymond Green,1.610613e+09,Golden State,Warriors,GSW,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,517.0,2024-25,3P,True


In [26]:
np.unique(streaks_df.PERIOD)

array([ 1.,  2.,  3.,  4.,  5.,  6., nan])

In [11]:
# Replace NaN rows with a unique marker (for example, 'NaN' string or a number)
streaks_df['streak_marker'] = streaks_df['PLAYER1_ID'].isna()

# Create a 'streak_id' column to identify the streak group
streaks_df['streak_id'] = streaks_df['streak_marker'].cumsum()

# Remove the NaN rows (streak separators) from the DataFrame
streaks_without_nans = streaks_df.dropna(subset=['PLAYER1_ID'])

# Count the lengths of each streak
streak_lengths = streaks_without_nans.groupby('streak_id').size()

# Filter streaks that are longer than X
X = 4 # Example streak length threshold
long_streaks_count = streak_lengths[streak_lengths > X].count()

# Output the count of long streaks
print(f"Number of streaks longer than {X}: {long_streaks_count}")

Number of streaks longer than 4: 1


In [12]:
def streak_effect(start_margin, end_margin):
    # Ensure both start_margin and end_margin are integers
    if not (isinstance(start_margin, int) and isinstance(end_margin, int)):
        raise ValueError("Both start_margin and end_margin must be integers.")

    # Compare when both margins are integers
    if start_margin > 0 and end_margin > 0:
        return "Continued leading" if end_margin > start_margin else "Narrowed lead"
    elif start_margin < 0 and end_margin < 0:
        return "Continued losing" if end_margin < start_margin else "Narrowed deficit"
    elif start_margin > 0 and end_margin < 0:
        return "From leading to losing"
    elif start_margin < 0 and end_margin > 0:
        return "From losing to leading"
    elif start_margin == 0 and end_margin != 0:
        return "Went from tie to leading" if end_margin > 0 else "Went from tie to losing"
    elif start_margin != 0 and end_margin == 0:
        return "Went from leading to tie" if start_margin > 0 else "Went from losing to tie"

    return "Stayed tied"  # When start_margin == 0 and end_margin == 0
    
    return "Unknown transition"

In [27]:
# Replace NaN rows with a unique marker
streaks_df['streak_marker'] = streaks_df['PLAYER1_ID'].isna()

# Create a 'streak_id' column to identify the streak group
streaks_df['streak_id'] = streaks_df['streak_marker'].cumsum()

# Remove the NaN rows (streak separators) from the DataFrame
streaks_without_nans = streaks_df.dropna(subset=['PLAYER1_ID'])

# Group by 'streak_id' and aggregate the length, list of games, and period
streak_summary = streaks_without_nans.groupby('streak_id').agg(
    length_of_streak=('PLAYER1_ID', 'size'),  # Count the length of each streak
    list_of_games=('GAME_ID', lambda x: x.unique().tolist()),  # Get the unique list of games for each streak
    start_margin=('SCOREMARGIN', 'first'),  # Get the starting score margin
    end_margin=('SCOREMARGIN', 'last'),  # Get the ending score margin
    period=('PERIOD', lambda x: x.iloc[-1])  # Get the last period before the streak ends
).reset_index(drop=True)

streak_summary['start_margin'] = streak_summary['start_margin'].replace('TIE', 0).astype(int)
streak_summary['end_margin'] = streak_summary['end_margin'].replace('TIE', 0).astype(int)

# Apply the effect function to the streak summary
streak_summary['effect'] = streak_summary.apply(
    lambda row: streak_effect(row['start_margin'], row['end_margin']), axis=1
)

# Filter to only include streaks with a length of 2 or greater
streak_summary = streak_summary[streak_summary['length_of_streak'] >= 2]

# Display the DataFrame
streak_summary

,length_of_streak,list_of_games,start_margin,end_margin,period,effect
0,2,[22200002.0],20,23,3.0,Continued leading
1,2,[22200002.0],16,16,4.0,Narrowed lead
4,3,[22200026.0],-6,-7,4.0,Continued losing
6,2,[22200042.0],4,7,2.0,Continued leading
17,2,[22200068.0],4,0,3.0,Went from leading to tie
...,...,...,...,...,...,...
513,2,[22301214.0],-7,-7,3.0,Narrowed deficit
519,2,[22301222.0],-3,2,5.0,From losing to leading
524,2,[22400007.0],-1,4,4.0,From losing to leading
525,2,[22400019.0],9,12,3.0,Continued leading


In [15]:
streak_summary.sort_values(by=['list_of_games'])

,length_of_streak,list_of_games,start_margin,end_margin,effect
0,2,[22200002.0],20,23,Continued leading
1,2,[22200002.0],16,16,Narrowed lead
4,3,[22200026.0],-6,-7,Continued losing
6,2,[22200042.0],4,7,Continued leading
17,2,[22200068.0],4,0,Went from leading to tie
...,...,...,...,...,...
513,2,[22301214.0],-7,-7,Narrowed deficit
519,2,[22301222.0],-3,2,From losing to leading
524,2,[22400007.0],-1,4,From losing to leading
525,2,[22400019.0],9,12,Continued leading


In [28]:
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2

steph_stats = pd.DataFrame()
games_stats = pd.DataFrame()

streak_summary = streak_summary.sort_values(by=['list_of_games'])

for _, row in tqdm(streak_summary.iterrows(), total=len(streak_summary)):
    games = row.list_of_games
    length_of_streak = row.length_of_streak
    effect = row.effect
    period = row.period

    for game in games:
        game_id = f"{int(game):010}"

        # Check if 'GAME_ID' column exists before accessing it
        if 'GAME_ID' in steph_stats.columns:
            # Check if the game already exists in the DataFrame
            if game_id in steph_stats['GAME_ID'].values:
                # Append the length of the streak to the existing list
                steph_stats.loc[steph_stats['GAME_ID'] == game_id, 'LENGTH_OF_STREAK'].apply(lambda x: x.append(length_of_streak))
                steph_stats.loc[steph_stats['GAME_ID'] == game_id, 'PERIOD'].apply(lambda x: x.append(period))

                continue  # Skip to the next game after appending

        boxscore = BoxScoreTraditionalV2(game_id=game_id, timeout=60).get_data_frames()
        time.sleep(0.5)

        basic_boxscore = boxscore[0]
        team_boxscore = boxscore[1]

        gsw_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION == 'GSW'].PTS.values[0]
        opponent_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION != 'GSW'].PTS.values[0]
        gsw_win = gsw_points > opponent_points

        new_row = basic_boxscore[basic_boxscore.PLAYER_ID == player_id].copy()
        new_row['LENGTH_OF_STREAK'] = [[length_of_streak]]
        new_row['PERIOD'] = [[period]]

        new_row['GAME_WON'] = gsw_win
        new_row['EFFECT'] = effect

        steph_stats = pd.concat([steph_stats, new_row], ignore_index=True)

    time.sleep(0.5)

100%|██████████| 92/92 [03:21<00:00,  2.19s/it]


In [42]:
steph_stats

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,LENGTH_OF_STREAK,PERIOD,GAME_WON,EFFECT
0,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,32.000000:48,10.0,22.0,0.455,4.0,13.0,0.308,9.0,9.0,1.000,1.0,5.0,6.0,7.0,4.0,1.0,4.0,1.0,33.0,20.0,"[2, 2]","[3.0, 4.0]",True,Continued leading
1,0022200026,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,35.000000:55,10.0,22.0,0.455,5.0,11.0,0.455,9.0,9.0,1.000,1.0,4.0,5.0,4.0,0.0,0.0,3.0,2.0,34.0,9.0,[3],[4.0],False,Continued losing
2,0022200042,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,31.000000:21,11.0,22.0,0.500,7.0,12.0,0.583,4.0,4.0,1.000,0.0,5.0,5.0,2.0,1.0,0.0,2.0,3.0,33.0,19.0,[2],[2.0],True,Continued leading
3,0022200068,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,36.000000:56,13.0,22.0,0.591,7.0,14.0,0.500,0.0,0.0,0.000,1.0,6.0,7.0,9.0,1.0,0.0,3.0,0.0,33.0,13.0,[2],[3.0],True,Went from leading to tie
4,0022200118,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,38.000000:21,13.0,22.0,0.591,8.0,15.0,0.533,5.0,5.0,1.000,1.0,2.0,3.0,9.0,0.0,0.0,5.0,3.0,39.0,4.0,"[2, 2]","[2.0, 3.0]",False,Continued losing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0022301214,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,34.000000:29,8.0,18.0,0.444,5.0,13.0,0.385,10.0,11.0,0.909,0.0,5.0,5.0,1.0,0.0,0.0,4.0,2.0,31.0,4.0,[2],[3.0],True,Narrowed deficit
68,0022301222,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,40.000000:22,10.0,20.0,0.500,6.0,12.0,0.500,8.0,8.0,1.000,0.0,6.0,6.0,3.0,1.0,0.0,3.0,0.0,34.0,-5.0,[2],[5.0],False,From losing to leading
69,0022400007,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,34.000000:52,14.0,27.0,0.519,5.0,12.0,0.417,4.0,5.0,0.800,0.0,6.0,6.0,9.0,1.0,2.0,4.0,2.0,37.0,24.0,[2],[4.0],True,From losing to leading
70,0022400019,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,26.000000:01,4.0,9.0,0.444,3.0,7.0,0.429,2.0,2.0,1.000,1.0,7.0,8.0,5.0,4.0,0.0,3.0,2.0,13.0,9.0,[2],[3.0],True,Continued leading


In [ ]:
steph_stats = pd.DataFrame()
games_stats = pd.DataFrame()

streak_summary = streak_summary.sort_values(by=['list_of_games'])

for _, row in tqdm(streak_summary.iterrows(), total=len(streak_summary)):
    games = row.list_of_games
    length_of_streak = row.length_of_streak
    effect = row.effect
    period = row.period

    for game in games:
        game_id = f"{int(game):010}"

        # Check if 'GAME_ID' column exists before accessing it
        if 'GAME_ID' in steph_stats.columns:
            # Check if the game already exists in the DataFrame
            if game_id in steph_stats['GAME_ID'].values:
                # Append the length of the streak to the existing list
                steph_stats.loc[steph_stats['GAME_ID'] == game_id, 'LENGTH_OF_STREAK'].apply(lambda x: x.append(length_of_streak))
                steph_stats.loc[steph_stats['GAME_ID'] == game_id, 'PERIOD'].apply(lambda x: x.append(period))

                continue  # Skip to the next game after appending

        boxscore = BoxScoreTraditionalV2(game_id=game_id, timeout=60).get_data_frames()
        time.sleep(0.5)

        basic_boxscore = boxscore[0]
        team_boxscore = boxscore[1]

        gsw_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION == 'GSW'].PTS.values[0]
        opponent_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION != 'GSW'].PTS.values[0]
        gsw_win = gsw_points > opponent_points

        new_row = basic_boxscore[basic_boxscore.PLAYER_ID == player_id].copy()
        new_row['LENGTH_OF_STREAK'] = [[length_of_streak]]
        new_row['PERIOD'] = [[period]]

        new_row['GAME_WON'] = gsw_win
        new_row['EFFECT'] = effect

        steph_stats = pd.concat([steph_stats, new_row], ignore_index=True)

    time.sleep(0.5)

In [75]:
import numpy as np

avg_points = np.mean(steph_stats.PTS)
avg_reb = np.mean(steph_stats.REB)
avg_ast = np.mean(steph_stats.AST)

avg_fg_perc = np.sum(steph_stats.FGM) / np.sum(steph_stats.FGA)
avg_3p_perc = np.sum(steph_stats.FG3M) / np.sum(steph_stats.FG3A)

avg_pm = np.mean(steph_stats.PLUS_MINUS)
win_percentage = (np.sum(steph_stats.GAME_WON) / len(steph_stats.GAME_WON)) * 100

print(f'GAMES - {len(steph_stats)}')
print(f'PTS: {avg_points:.2f}')
print(f'REB: {avg_reb:.2f}')
print(f'AST: {avg_ast:.2f}')
print(f'FG%: {avg_fg_perc*100:.2f}%')
print(f'3P%: {avg_3p_perc*100:.2f}%')
print(f'Win%: {win_percentage:.2f}%')
print(f'PM: {avg_pm:.2f}')

GAMES - 72
PTS: 31.25
REB: 5.32
AST: 5.96
FG%: 50.53%
3P%: 47.14%
Win%: 65.28%
PM: 6.96


In [45]:
def get_quarter_times(quarter):
    quarter_times = {
        1: [0, 7200],              # Q1: Start at 0, end at 7200
        2: [7205, 14405],          # Q2: Start at 7205, end at 14405
        3: [14410, 21610],         # Q3: Start at 14410, end at 21610
        4: [21615, 28815],         # Q4: Start at 21615, end at 28815
        5: [28820, 31820],         # OT1: Start at 28820, end at 31820 (5-minute OT)
        6: [31825, 34825]          # OT2: Start at 31825, end at 34825 (5-minute OT)
    }
    
    return tuple(quarter_times.get(quarter, (None, None)))

In [62]:
# Initialize an empty DataFrame for the new detailed quarterly stats
quarterly_stats = pd.DataFrame()

# Iterate over each row in the steph_stats DataFrame
for _, row in tqdm(steph_stats.iterrows(), total=len(steph_stats)):
    game_id = row['GAME_ID']
    player_id = row['PLAYER_ID']
    length_of_streak_list = row['LENGTH_OF_STREAK']  # List of streak lengths
    periods_list = row['PERIOD']  # List of quarters where the streak occurred
    gsw_win = row['GAME_WON']
    effect = row['EFFECT']
    

    # Get the box score for the game for each period
    for i, period in enumerate(periods_list):
        # Get the start and end times for the given period
        start, end = get_quarter_times(period)

        # Fetch the boxscore for the specific quarter
        boxscore = BoxScoreTraditionalV2(game_id=game_id,
                                         start_range=start,
                                         end_range=end,
                                         range_type=2,
                                         timeout=60
                                         ).get_data_frames()
        time.sleep(0.5)

        # Extract the detailed box score for all players
        basic_boxscore = boxscore[0]

        # Filter for the player of interest (Steph Curry)
        player_boxscore = basic_boxscore[basic_boxscore['PLAYER_ID'] == player_id].copy()  # Make a copy

        # Safely modify the 'PERIOD' and 'LENGTH_OF_STREAK' columns
        player_boxscore.loc[:, 'PERIOD'] = period
        player_boxscore.loc[:, 'LENGTH_OF_STREAK'] = length_of_streak_list[i]
        player_boxscore.loc[:, 'GAME_WON'] = gsw_win
        player_boxscore.loc[:, 'EFFECT'] = effect


        # Append this row to the new DataFrame
        quarterly_stats = pd.concat([quarterly_stats, player_boxscore], ignore_index=True)
    time.sleep(0.5)


100%|██████████| 72/72 [01:35<00:00,  1.32s/it]


In [69]:
print(set(sorted(quarterly_stats.PERIOD.to_list())))

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0}


In [74]:
periods = quarterly_stats.PERIOD.to_list()

for period in set(sorted(quarterly_stats.PERIOD.to_list())):
    # Filter rows where the streaks occurred in the current period
    period_stat = quarterly_stats[quarterly_stats['PERIOD'] == period]

    if not period_stat.empty:
        # Calculate average stats for the specific period
        avg_points = np.mean(period_stat.PTS)
        avg_reb = np.mean(period_stat.REB)
        avg_ast = np.mean(period_stat.AST)

        avg_fg_perc = np.sum(period_stat.FGM) / np.sum(period_stat.FGA)
        avg_3p_perc = np.sum(period_stat.FG3M) / np.sum(period_stat.FG3A)

        avg_pm = np.mean(period_stat.PLUS_MINUS)
        win_percentage = (np.sum(period_stat.GAME_WON) / len(period_stat.GAME_WON)) * 100
        effect_counts = period_stat['EFFECT'].value_counts()

        # Count the streaks of each length that occurred in this period
        streak_lengths_in_period = period_stat['LENGTH_OF_STREAK'].explode().value_counts().sort_index()
        if period == 5:
            text = 'FIRST OVERTIME'
        elif period == 6:
            text = 'SECOND OVERTIME'
        else:
            text = f'Q{int(period)}'
        print(text)
        print(f'NUMBER OF GAMES: {len(period_stat)}')
        print(f'	PTS: {avg_points:.1f}')
        print(f'	REB: {avg_reb:.1f}')
        print(f'	AST: {avg_ast:.1f}')
        print(f'	FG%: {avg_fg_perc*100:.1f}%')
        print(f'	3P%: {avg_3p_perc*100:.1f}%')
        print(f'	PM: {avg_pm:.1f}')
        print(f'	Win%: {win_percentage:.1f}%')
        print(f'	Effect:')
        for effect, count in effect_counts.items():
            print(f'		{effect}: {count} times')

        # Output streak lengths in this period
        print(f'	Streak lengths in {text}:')
        for streak_length, count in streak_lengths_in_period.items():
            print(f'		Streak Length {streak_length}: {count} times')
        print()

Q1
NUMBER OF GAMES: 21
	PTS: 11.1
	REB: 2.1
	AST: 1.7
	FG%: 64.8%
	3P%: 68.2%
	PM: 6.3
	Win%: 71.4%
	Effect:
		Continued leading: 7 times
		Narrowed lead: 3 times
		Narrowed deficit: 3 times
		Continued losing: 3 times
		From losing to leading: 3 times
		Stayed tied: 2 times
	Streak lengths in Q1:
		Streak Length 2: 17 times
		Streak Length 3: 4 times

Q2
NUMBER OF GAMES: 16
	PTS: 10.8
	REB: 1.4
	AST: 1.0
	FG%: 64.8%
	3P%: 70.2%
	PM: 1.3
	Win%: 50.0%
	Effect:
		Continued losing: 4 times
		Narrowed deficit: 4 times
		Continued leading: 3 times
		Narrowed lead: 3 times
		Went from tie to leading: 1 times
		From losing to leading: 1 times
	Streak lengths in Q2:
		Streak Length 2: 13 times
		Streak Length 3: 3 times

Q3
NUMBER OF GAMES: 29
	PTS: 11.7
	REB: 1.8
	AST: 1.5
	FG%: 57.1%
	3P%: 55.2%
	PM: 2.2
	Win%: 65.5%
	Effect:
		Continued leading: 9 times
		Narrowed deficit: 7 times
		Narrowed lead: 5 times
		Continued losing: 3 times
		Went from leading to tie: 2 times
		From leading to losi

In [32]:
# Assuming steph_stats['LENGTH_OF_STREAK'] now contains lists of streaks, e.g., [2, 3]
# Iterate through each unique streak length (2 to 5)
for streak_length in range(2, 6):
    # Filter rows where the streak list contains the current streak length
    streak_stat = steph_stats[steph_stats['LENGTH_OF_STREAK'].apply(lambda x: streak_length in x)]

    if not streak_stat.empty:
        avg_points = np.mean(streak_stat.PTS)
        avg_reb = np.mean(streak_stat.REB)
        avg_ast = np.mean(streak_stat.AST)

        avg_fg_perc = np.sum(streak_stat.FGM) / np.sum(streak_stat.FGA)
        avg_3p_perc = np.sum(streak_stat.FG3M) / np.sum(streak_stat.FG3A)

        avg_pm = np.mean(streak_stat.PLUS_MINUS)
        win_percentage = (np.sum(streak_stat.GAME_WON) / len(streak_stat.GAME_WON)) * 100
        effect_counts = streak_stat['EFFECT'].value_counts()

        # Identify the periods in which the streak occurred
        periods_in_streak = streak_stat['PERIOD'].explode().value_counts().sort_index()

        print(f'STREAK LENGTH : {streak_length}')
        print(f'NUMBER OF STREAKS: {len(streak_summary[streak_summary.length_of_streak == streak_length])}')
        print(f'NUMBER OF GAMES: {len(streak_stat)}')
        print(f'	PTS: {avg_points:.1f}')
        print(f'	REB: {avg_reb:.1f}')
        print(f'	AST: {avg_ast:.1f}')
        print(f'	FG%: {avg_fg_perc*100:.1f}%')
        print(f'	3P%: {avg_3p_perc*100:.1f}%')
        print(f'	PM: {avg_pm:.1f}')
        print(f'	Win%: {win_percentage:.1f}%')
        print(f'	Effect:')
        for effect, count in effect_counts.items():
            print(f'		{effect}: {count} times')
        print(f'	Periods where streaks occurred:')
        for period, count in periods_in_streak.items():
            print(f'		Period {period}: {count} times')
        print()

STREAK LENGTH : 2
NUMBER OF STREAKS: 75
NUMBER OF GAMES: 58
	PTS: 30.7
	REB: 5.3
	AST: 6.2
	FG%: 51.4%
	3P%: 48.0%
	PM: 8.5
	Win%: 67.2%
	Effect:
		Continued leading: 17 times
		Continued losing: 10 times
		Narrowed deficit: 10 times
		Narrowed lead: 9 times
		From losing to leading: 5 times
		Went from leading to tie: 2 times
		From leading to losing: 2 times
		Went from tie to leading: 2 times
		Stayed tied: 1 times
	Periods where streaks occurred:
		Period 1.0: 18 times
		Period 2.0: 14 times
		Period 3.0: 28 times
		Period 4.0: 17 times
		Period 5.0: 1 times
		Period 6.0: 1 times

STREAK LENGTH : 3
NUMBER OF STREAKS: 15
NUMBER OF GAMES: 15
	PTS: 33.0
	REB: 5.3
	AST: 5.1
	FG%: 48.4%
	3P%: 46.1%
	PM: -0.3
	Win%: 53.3%
	Effect:
		Narrowed deficit: 6 times
		Continued losing: 3 times
		Narrowed lead: 2 times
		Continued leading: 2 times
		Went from losing to tie: 1 times
		Stayed tied: 1 times
	Periods where streaks occurred:
		Period 1.0: 5 times
		Period 2.0: 4 times
		Period 3.0: 4 

In [25]:
num_rows_with_streak_2 = steph_stats['LENGTH_OF_STREAK'].apply(lambda x: 2 in x)

steph_stats[num_rows_with_streak_2]


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,LENGTH_OF_STREAK,GAME_WON,EFFECT
0,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,32.000000:48,10.0,22.0,0.455,4.0,13.0,0.308,9.0,9.0,1.000,1.0,5.0,6.0,7.0,4.0,1.0,4.0,1.0,33.0,20.0,"[2, 2]",True,Continued leading
2,0022200042,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,31.000000:21,11.0,22.0,0.500,7.0,12.0,0.583,4.0,4.0,1.000,0.0,5.0,5.0,2.0,1.0,0.0,2.0,3.0,33.0,19.0,[2],True,Continued leading
3,0022200068,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,36.000000:56,13.0,22.0,0.591,7.0,14.0,0.500,0.0,0.0,0.000,1.0,6.0,7.0,9.0,1.0,0.0,3.0,0.0,33.0,13.0,[2],True,Went from leading to tie
4,0022200118,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,38.000000:21,13.0,22.0,0.591,8.0,15.0,0.533,5.0,5.0,1.000,1.0,2.0,3.0,9.0,0.0,0.0,5.0,3.0,39.0,4.0,"[2, 2]",False,Continued losing
5,0022200156,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,37.000000:43,17.0,24.0,0.708,7.0,12.0,0.583,6.0,7.0,0.857,0.0,8.0,8.0,8.0,0.0,0.0,0.0,2.0,47.0,20.0,[2],True,Narrowed deficit
6,0022200182,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,34.000000:00,15.0,23.0,0.652,6.0,11.0,0.545,4.0,6.0,0.667,0.0,4.0,4.0,5.0,1.0,0.0,2.0,4.0,40.0,7.0,"[2, 2]",True,Narrowed deficit
7,0022200233,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,33.000000:50,9.0,19.0,0.474,5.0,13.0,0.385,1.0,3.0,0.333,0.0,6.0,6.0,10.0,2.0,0.0,5.0,2.0,24.0,18.0,[2],True,Continued leading
8,0022200245,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,36.000000:60,11.0,20.0,0.550,7.0,14.0,0.500,4.0,5.0,0.800,0.0,6.0,6.0,15.0,3.0,2.0,1.0,3.0,33.0,22.0,[2],True,From leading to losing
9,0022200271,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,31.000000:41,9.0,20.0,0.450,4.0,10.0,0.400,0.0,0.0,0.000,1.0,5.0,6.0,9.0,0.0,0.0,1.0,0.0,22.0,18.0,[2],True,Continued leading
10,0022200284,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,35.000000:43,13.0,23.0,0.565,6.0,13.0,0.462,1.0,1.0,1.000,0.0,5.0,5.0,4.0,2.0,0.0,5.0,4.0,33.0,19.0,[2],True,Continued leading


In [80]:
np.unique(df[df.PLAYER1_ID == player_id].PERIOD).tolist()

[1, 2, 3, 4, 5, 6]

In [107]:
steph_games = np.unique(df[df.PLAYER1_ID == player_id].GAME_ID)
periods = np.unique(df[df.PLAYER1_ID == player_id].PERIOD).tolist()
steph_stats_quarters = pd.DataFrame()

whole_game_steph_stats = pd.DataFrame()

for game_id in tqdm(steph_games, total=len(steph_games)):
    game_id = f"{int(game_id):010}"

    whole_game_boxscore = BoxScoreTraditionalV2(game_id=game_id, timeout=60).get_data_frames()
    time.sleep(0.5)
    
    player_boxscore = whole_game_boxscore[0]
    team_boxscore = whole_game_boxscore[1]
    
    whole_player_boxscore = player_boxscore[player_boxscore['PLAYER_ID'] == player_id].copy()  # Make a copy

    gsw_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION == 'GSW'].PTS.values[0]
    opponent_points = team_boxscore[team_boxscore.TEAM_ABBREVIATION != 'GSW'].PTS.values[0]
    gsw_win = gsw_points > opponent_points
    
    whole_player_boxscore['GAME_WON'] = gsw_win
    
    whole_game_steph_stats = pd.concat([whole_game_steph_stats, whole_player_boxscore])
    
    for period in periods:
        start, end = get_quarter_times(period)
    
        quarter_boxscore = BoxScoreTraditionalV2(game_id=game_id,
                                         start_range=start,
                                         end_range=end,
                                         range_type=2,
                                         timeout=60
                                         ).get_data_frames()
        
        time.sleep(0.5)

        # Extract the detailed box score for all players
        basic_boxscore = quarter_boxscore[0]

        # Filter for the player of interest (Steph Curry)
        quarter_player_boxscore = basic_boxscore[basic_boxscore['PLAYER_ID'] == player_id].copy()  # Make a copy
        
        quarter_player_boxscore['PERIOD'] = period
        quarter_player_boxscore['GAME_WON'] = gsw_win
        
        steph_stats_quarters = pd.concat([steph_stats_quarters, quarter_player_boxscore], ignore_index=True)
    
    time.sleep(0.5)

100%|██████████| 139/139 [54:35<00:00, 23.56s/it]


In [109]:
steph_stats_quarters

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,PERIOD,GAME_WON
0,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,9.000000:20,1,7,0.143,0,4,0.000,2,2,1.0,1,4,5,2,2,0,2,1,4,0.0,1,True
1,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,8.000000:18,4,6,0.667,0,1,0.000,4,4,1.0,0,0,0,2,0,0,0,0,12,2.0,2,True
2,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,10.000000:27,2,4,0.500,2,4,0.500,2,2,1.0,0,1,1,1,2,1,1,0,8,18.0,3,True
3,0022200002,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,4.000000:43,3,5,0.600,2,4,0.500,1,1,1.0,0,0,0,2,0,0,1,0,9,0.0,4,True
4,0022200026,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,9.000000:29,2,4,0.500,1,3,0.333,4,4,1.0,0,1,1,2,0,0,0,0,9,1.0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,0022400188,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,7.000000:58,2,4,0.500,0,1,0.000,1,1,1.0,0,1,1,2,0,0,2,0,5,4.0,3,False
546,0022400207,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,9.000000:21,2,5,0.400,1,3,0.333,0,0,0.0,1,1,2,2,0,0,1,1,5,-2.0,1,True
547,0022400207,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,7.000000:53,2,4,0.500,2,2,1.000,0,1,0.0,0,0,0,2,1,0,0,1,6,8.0,2,True
548,0022400207,1610612744,GSW,Golden State,201939,Stephen Curry,Stephen,G,,10.000000:42,7,10,0.700,2,4,0.500,1,1,1.0,0,3,3,2,0,0,1,0,17,21.0,3,True


In [121]:
# Iterate over the number of 3-pointers made
for num_threes in range(2, quarterly_stats['FG3M'].max() + 1):
    num_threes_data = quarterly_stats[quarterly_stats['FG3M'] == num_threes]
    print(f'Steph hits {num_threes} threes in a quarter')
    if not num_threes_data.empty:
        for period in range(1, 7):
            print(f'Q{period}')
            period_data = num_threes_data[num_threes_data['PERIOD'] == period]
                        
            if not period_data.empty:
                avg_points = np.mean(period_data.PTS)
                avg_reb = np.mean(period_data.REB)
                avg_ast = np.mean(period_data.AST)
                
                avg_fg_perc = np.sum(period_data.FGM) / np.sum(period_data.FGA) if np.sum(period_data.FGA) > 0 else 0
                avg_3p_perc = np.sum(period_data.FG3M) / np.sum(period_data.FG3A) if np.sum(period_data.FG3A) > 0 else 0

                avg_pm = np.mean(period_data.PLUS_MINUS)
                win_percentage = (np.sum(period_data.GAME_WON) / len(period_data.GAME_WON)) * 100

                print(f'    PTS: {avg_points:.1f}')
                print(f'    REB: {avg_reb:.1f}')
                print(f'    AST: {avg_ast:.1f}')
                print(f'    FG%: {avg_fg_perc*100:.1f}%')
                print(f'    3P%: {avg_3p_perc*100:.1f}%')
                print(f'    PM: {avg_pm:.1f}')
                print(f'    Win%: {win_percentage:.1f}%')
                print()

Steph hits 2 threes in a quarter
Q1
    PTS: 8.0
    REB: 2.2
    AST: 1.8
    FG%: 58.1%
    3P%: 63.2%
    PM: 6.7
    Win%: 83.3%

Q2
    PTS: 8.8
    REB: 1.6
    AST: 1.5
    FG%: 58.8%
    3P%: 64.5%
    PM: 1.4
    Win%: 40.0%

Q3
    PTS: 9.6
    REB: 1.7
    AST: 1.0
    FG%: 48.0%
    3P%: 44.0%
    PM: 1.8
    Win%: 54.5%

Q4
    PTS: 9.8
    REB: 0.5
    AST: 0.9
    FG%: 68.6%
    3P%: 61.1%
    PM: 1.3
    Win%: 90.9%

Q5
    PTS: 9.0
    REB: 0.5
    AST: 0.0
    FG%: 75.0%
    3P%: 66.7%
    PM: -4.5
    Win%: 0.0%

Q6
    PTS: 10.0
    REB: 0.0
    AST: 0.0
    FG%: 60.0%
    3P%: 66.7%
    PM: -1.0
    Win%: 0.0%

Steph hits 3 threes in a quarter
Q1
    PTS: 12.2
    REB: 2.7
    AST: 2.2
    FG%: 68.5%
    3P%: 69.2%
    PM: 8.6
    Win%: 66.7%

Q2
    PTS: 12.6
    REB: 1.4
    AST: 0.2
    FG%: 72.4%
    3P%: 75.0%
    PM: -1.8
    Win%: 60.0%

Q3
    PTS: 14.6
    REB: 1.5
    AST: 1.9
    FG%: 65.6%
    3P%: 63.5%
    PM: 1.3
    Win%: 63.6%

Q4
    PTS: 13.2
   

In [119]:
for num_threes in range(2, quarterly_stats['FG3M'].max() + 1):
    games_with_threes = quarterly_stats[quarterly_stats['FG3M'] == num_threes]
    game_ids_game_with_threes = games_with_threes.GAME_ID
    game_ids_game_with_threes = np.unique(game_ids_game_with_threes)
    
    data = whole_game_steph_stats[whole_game_steph_stats.GAME_ID.isin(game_ids_game_with_threes)]
    
    if not data.empty:
        avg_points = np.mean(data.PTS)
        avg_reb = np.mean(data.REB)
        avg_ast = np.mean(data.AST)
        
        avg_fg_perc = np.sum(data.FGM) / np.sum(data.FGA) if np.sum(data.FGA) > 0 else 0
        avg_3p_perc = np.sum(data.FG3M) / np.sum(data.FG3A) if np.sum(data.FG3A) > 0 else 0

        avg_pm = np.mean(data.PLUS_MINUS)
        win_percentage = (np.sum(data.GAME_WON) / len(data.GAME_WON)) * 100
        
        print(f'Steph hits {num_threes} threes in a quarter')
        print(f'    NUMBER OF GAMES: {len(game_ids_game_with_threes)}')
        print(f'    PTS: {avg_points:.1f}')
        print(f'    REB: {avg_reb:.1f}')
        print(f'    AST: {avg_ast:.1f}')
        print(f'    FG%: {avg_fg_perc*100:.1f}%')
        print(f'    3P%: {avg_3p_perc*100:.1f}%')
        print(f'    PM: {avg_pm:.1f}')
        print(f'    Win%: {win_percentage:.1f}%')
        print()

Steph hits 2 threes in a quarter
    NUMBER OF GAMES: 39
    PTS: 31.2
    REB: 5.5
    AST: 5.9
    FG%: 51.4%
    3P%: 46.4%
    PM: 6.5
    Win%: 59.0%

Steph hits 3 threes in a quarter
    NUMBER OF GAMES: 28
    PTS: 33.4
    REB: 5.2
    AST: 5.9
    FG%: 51.1%
    3P%: 48.7%
    PM: 5.5
    Win%: 67.9%

Steph hits 4 threes in a quarter
    NUMBER OF GAMES: 8
    PTS: 32.4
    REB: 5.4
    AST: 6.4
    FG%: 50.9%
    3P%: 51.8%
    PM: 9.9
    Win%: 87.5%

Steph hits 5 threes in a quarter
    NUMBER OF GAMES: 3
    PTS: 30.3
    REB: 5.7
    AST: 4.7
    FG%: 49.3%
    3P%: 47.6%
    PM: 13.7
    Win%: 66.7%


In [118]:
steph_stats_quarters.to_csv('steph_stats_per_quarter_2022-2024.csv')

In [120]:
whole_game_steph_stats.to_csv('steph_stats_game_2022-2024.csv')